In [19]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from env import *
from time import sleep
from prepare import *

pd.set_option("display.max_rows", None, "display.max_columns", None)

pd.reset_option("display.max_rows", "display.max_columns")

import re

In [2]:
names = pd.read_csv('more_players.csv')

In [3]:
names = names [['players']]

In [4]:
names

,players
0,Satoshi
1,Johnsun123
2,bogo binter
3,Sybr
4,good nightt
...,...
878,godsent mid
879,Alamander
880,hanason1
881,DabinPenguEmote


In [5]:
names[names.index==85]

,players
85,Lefasa


In [6]:
names

,players
0,Satoshi
1,Johnsun123
2,bogo binter
3,Sybr
4,good nightt
...,...
878,godsent mid
879,Alamander
880,hanason1
881,DabinPenguEmote


In [7]:
def get_puuid(name_list):
    puuid_list = []
    counter = 0
    for name in name_list:
        print(counter)
        counter += 1
        if counter%50 == 0:
            print('function paused')
            print
            time.sleep(180)
            print('Continue')
            response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}")
            summoner_info = response.json()
            puuid_list.append(summoner_info)
        else:
            response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}")
            summoner_info = response.json()
            puuid_list.append(summoner_info)
    return puuid_list

In [8]:
# puuid_list = get_puuid(names.players)

In [9]:
# puuid_list

In [10]:
# puuid_df = pd.DataFrame(puuid_list)

In [11]:
# puuid_df.drop(columns = 'status', inplace=True)

In [12]:
# puuid_df.dropna(inplace=True)

In [13]:
# puuid_df.to_csv('puuid.csv',index=False)

In [14]:
# puuid = puuid_df[['puuid']]

In [15]:
def get_match_info(match_ids, api_key, username, time = 20):
    """
    This function takes in a list of match ids and iterates through them. For each iteration,
    it will make two api calls and retrieve the necessary information for our project.
    
    Two json lists will be created, and once all the info has been gathered for each match id,
    this function will save both json lists for future reference. 
    
    This function also takes in a username string. This will be used for naming the saved files.
    
    Finally, this function will funnel the resulting json lists into a prepare function and
    return a complete df.
    
    The time parameter is used in the prepare function that will be called at the end
    and represents the timeframe of the data we want to gather and prepare.
    The default value is 20 for the 20 minute mark of the match.
    
    """
    
    #Create an empty list to store the timeline json data
    timeline_data = []
    
    #Create an empty list to store the other json data
    game_data = []
    
    #Set up main url
    url = 'https://americas.api.riotgames.com/'
    
    #Create a count var
    n = 0
    
    #Store the length of the match_ids list in a var
    match_count = len(match_ids)
    
    #Loop through each match_id
    for match in match_ids:
        #Update the count var
        n += 1
        
        #After every 50 match ids (100 total requests), wait 150 seconds (2.5 minutes)
        if n % 50 == 0:
            print(f'Completed {n} of {match_count} match IDs.')
            print(f'Waiting 150 seconds...\n')
            
            #Sleep 150 seconds
            sleep(150)
            
            print('Continuing!\n')
        
        ############################# TIMELINE DATA #######################
        #Set up timeline url
        timeline_query = f'lol/match/v5/matches/{match}/timeline/?api_key={api_key}'
        
        #Grab timeline json data
        timeline_response = requests.get(url + timeline_query)
        
        #Check response and leave a status message
        if timeline_response.status_code != 200:
            print(f'Something went wrong getting TIMELINE DATA! Status Code {game_response.status_code} for match ID: {match}.')
            print(f'Skipping this match ID.\n')
            continue

        #Turn it into json format
        timeline_json = timeline_response.json()
        
        #Append this data to the timeline_data list
        timeline_data.append(timeline_json)
        
        ############################# OTHER GAME DATA #######################
        #Set up game data url
        game_query = f'lol/match/v5/matches/{match}?api_key={api_key}'
        
        #Grab game json data
        game_response = requests.get(url + game_query)
        
        #Check response and leave a status message.
        if game_response.status_code != 200:
            print(f'Something went wrong getting OTHER GAME DATA! Status Code {game_response.status_code} for match ID: {match}.')
            print(f'Skipping this match ID and REMOVING PREVIOUS TIMELINE ENTRY.\n')
            
            #Remove the last entry in the timeline_data list
            timeline_data.pop()
            continue
        
        #Turn it into json format
        game_json = game_response.json()
        
        #Append this data to the game_data list
        game_data.append(game_json)
        
    ################################## END LOOP ##############################
    
    #Now that all of the json data has been gathered, save each file for future reference
    
    #Save timeline data.
    #First, convert it to a df
    timeline_df = pd.DataFrame(timeline_data)
    
    #Save as csv file. Use username string to identify whose file it is
    timeline_df.to_json(f'timeline_data_{username}.json')
    
    #Print status message
    print(f'Created timeline_data_{username}.json')
    
    #Save other game data
    #First, convert it to a df
    game_df = pd.DataFrame(game_data)
    
    #Save as csv file. Use username string to identify whose file it is
    game_df.to_json(f'other_game_data_{username}.json')
    
    #Print status message
    print(f'Created other_game_data_{username}.json\n')
    
    ################################ PREPARE DATA #########################
    
    #The following section will funnel the data into the prepare function
    #Written by Joshua C.
    
    #Leave a status message
    print('Preparing the data...\n')
    
    #Begin preparing the data
    df = prepare.prepare(timeline_data, game_data, time)
    
    #Finally, return the prepared df
    return df

In [16]:
match_ids = pd.read_csv('match_ids.csv')

In [17]:
match_ids = list(match_ids['0'])[1000:2000]

In [18]:
df = get_match_info(match_ids, api_key, username, time=20)

Completed 50 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 100 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 150 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 200 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 250 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 300 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 350 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 400 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 450 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 500 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 550 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 600 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 650 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 700 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 750 of 1000

NameError: name 'prepare' is not defined

In [21]:
other_game_data = pd.read_json('other_game_data_bryant.json')

In [22]:
timeline_data = pd.read_json('timeline_data_bryant.json')

In [23]:
timeline_list = timeline_data.to_dict(orient = 'records')

In [25]:
other_game_list = other_game_data.to_dict(orient = 'records')

In [27]:
df = prepare(timeline_list, other_game_list, time=20)

Finished with: 0 of 1000
Skipping: 1 due to <20 min or not classic
Skipping: 2 due to <20 min or not classic
Skipping: 3 due to <20 min or not classic
Finished with: 4 of 1000
Finished with: 5 of 1000
Finished with: 6 of 1000
Skipping: 7 due to <20 min or not classic
Finished with: 8 of 1000
Skipping: 9 due to <20 min or not classic
Finished with: 10 of 1000
Skipping: 11 due to <20 min or not classic
Finished with: 12 of 1000
Finished with: 13 of 1000
Skipping: 14 due to <20 min or not classic
Finished with: 15 of 1000
Finished with: 16 of 1000
Skipping: 17 due to <20 min or not classic
Skipping: 18 due to <20 min or not classic
Finished with: 19 of 1000
Skipping: 20 due to <20 min or not classic
Finished with: 21 of 1000
Finished with: 22 of 1000
Skipping: 23 due to <20 min or not classic
Finished with: 24 of 1000
Skipping: 25 due to <20 min or not classic
Finished with: 26 of 1000
Finished with: 27 of 1000
Finished with: 28 of 1000
Finished with: 29 of 1000
Skipping: 30 due to <20 mi

Finished with: 263 of 1000
Finished with: 264 of 1000
Finished with: 265 of 1000
Finished with: 266 of 1000
Skipping: 267 due to <20 min or not classic
Finished with: 268 of 1000
Skipping: 269 due to <20 min or not classic
Finished with: 270 of 1000
Finished with: 271 of 1000
Skipping: 272 due to <20 min or not classic
Skipping: 273 due to <20 min or not classic
Finished with: 274 of 1000
Finished with: 275 of 1000
Finished with: 276 of 1000
Finished with: 277 of 1000
Skipping: 278 due to <20 min or not classic
Finished with: 279 of 1000
Finished with: 280 of 1000
Finished with: 281 of 1000
Finished with: 282 of 1000
Finished with: 283 of 1000
Finished with: 284 of 1000
Finished with: 285 of 1000
Finished with: 286 of 1000
Finished with: 287 of 1000
Finished with: 288 of 1000
Finished with: 289 of 1000
Finished with: 290 of 1000
Finished with: 291 of 1000
Skipping: 292 due to <20 min or not classic
Finished with: 293 of 1000
Finished with: 294 of 1000
Finished with: 295 of 1000
Skippin

Finished with: 522 of 1000
Finished with: 523 of 1000
Finished with: 524 of 1000
Finished with: 525 of 1000
Finished with: 526 of 1000
Skipping: 527 due to <20 min or not classic
Finished with: 528 of 1000
Finished with: 529 of 1000
Skipping: 530 due to <20 min or not classic
Finished with: 531 of 1000
Finished with: 532 of 1000
Finished with: 533 of 1000
Finished with: 534 of 1000
Skipping: 535 due to <20 min or not classic
Finished with: 536 of 1000
Finished with: 537 of 1000
Skipping: 538 due to <20 min or not classic
Finished with: 539 of 1000
Finished with: 540 of 1000
Finished with: 541 of 1000
Skipping: 542 due to <20 min or not classic
Skipping: 543 due to <20 min or not classic
Finished with: 544 of 1000
Finished with: 545 of 1000
Finished with: 546 of 1000
Skipping: 547 due to <20 min or not classic
Finished with: 548 of 1000
Finished with: 549 of 1000
Finished with: 550 of 1000
Finished with: 551 of 1000
Finished with: 552 of 1000
Finished with: 553 of 1000
Finished with: 55

Finished with: 784 of 1000
Finished with: 785 of 1000
Finished with: 786 of 1000
Finished with: 787 of 1000
Finished with: 788 of 1000
Finished with: 789 of 1000
Skipping: 790 due to <20 min or not classic
Finished with: 791 of 1000
Finished with: 792 of 1000
Finished with: 793 of 1000
Finished with: 794 of 1000
Finished with: 795 of 1000
Finished with: 796 of 1000
Finished with: 797 of 1000
Finished with: 798 of 1000
Finished with: 799 of 1000
Finished with: 800 of 1000
Skipping: 801 due to <20 min or not classic
Finished with: 802 of 1000
Finished with: 803 of 1000
Finished with: 804 of 1000
Finished with: 805 of 1000
Finished with: 806 of 1000
Finished with: 807 of 1000
Finished with: 808 of 1000
Skipping: 809 due to <20 min or not classic
Finished with: 810 of 1000
Skipping: 811 due to <20 min or not classic
Finished with: 812 of 1000
Finished with: 813 of 1000
Finished with: 814 of 1000
Finished with: 815 of 1000
Finished with: 816 of 1000
Finished with: 817 of 1000
Skipping: 818 

In [28]:
df.head()

,assistsplayer_1,assistsplayer_10,assistsplayer_2,assistsplayer_3,assistsplayer_5,assistsplayer_6,assistsplayer_7,assistsplayer_8,assistsplayer_9,currentGold_1,currentGold_10,currentGold_2,currentGold_3,currentGold_4,currentGold_5,currentGold_6,currentGold_7,currentGold_8,currentGold_9,deathsplayer_1,deathsplayer_2,deathsplayer_3,deathsplayer_4,deathsplayer_5,deathsplayer_6,deathsplayer_8,deathsplayer_9,dragon_team200,gameMode,gameType,gameVersion,goldPerSecond_1,goldPerSecond_10,goldPerSecond_2,goldPerSecond_3,goldPerSecond_4,goldPerSecond_5,goldPerSecond_6,goldPerSecond_7,goldPerSecond_8,goldPerSecond_9,inhibs_lost_team100,jungleMinionsKilled_1,jungleMinionsKilled_10,jungleMinionsKilled_2,jungleMinionsKilled_3,jungleMinionsKilled_4,jungleMinionsKilled_5,jungleMinionsKilled_6,jungleMinionsKilled_7,jungleMinionsKilled_8,jungleMinionsKilled_9,killsplayer_1,killsplayer_10,killsplayer_2,killsplayer_3,killsplayer_4,killsplayer_5,killsplayer_6,killsplayer_7,killsplayer_8,killsplayer_9,level_1,level_10,level_2,level_3,level_4,level_5,level_6,level_7,level_8,level_9,magicDamageDoneToChampions_1,magicDamageDoneToChampions_10,magicDamageDoneToChampions_2,magicDamageDoneToChampions_3,magicDamageDoneToChampions_4,magicDamageDoneToChampions_5,magicDamageDoneToChampions_6,magicDamageDoneToChampions_7,magicDamageDoneToChampions_8,magicDamageDoneToChampions_9,minionsKilled_1,minionsKilled_10,minionsKilled_2,minionsKilled_3,minionsKilled_4,minionsKilled_5,minionsKilled_6,minionsKilled_7,minionsKilled_8,minionsKilled_9,physicalDamageDoneToChampions_1,physicalDamageDoneToChampions_10,physicalDamageDoneToChampions_2,physicalDamageDoneToChampions_3,physicalDamageDoneToChampions_4,physicalDamageDoneToChampions_5,physicalDamageDoneToChampions_6,physicalDamageDoneToChampions_7,physicalDamageDoneToChampions_8,physicalDamageDoneToChampions_9,riftherald_team200,riftherald_team300,timeEnemySpentControlled_1,timeEnemySpentControlled_10,timeEnemySpentControlled_2,timeEnemySpentControlled_3,timeEnemySpentControlled_4,timeEnemySpentControlled_5,timeEnemySpentControlled_6,timeEnemySpentControlled_7,timeEnemySpentControlled_8,timeEnemySpentControlled_9,timestamp,totalDamageDoneToChampions_1,totalDamageDoneToChampions_10,totalDamageDoneToChampions_2,totalDamageDoneToChampions_3,totalDamageDoneToChampions_4,totalDamageDoneToChampions_5,totalDamageDoneToChampions_6,totalDamageDoneToChampions_7,totalDamageDoneToChampions_8,totalDamageDoneToChampions_9,totalGold_1,totalGold_10,totalGold_2,totalGold_3,totalGold_4,totalGold_5,totalGold_6,totalGold_7,totalGold_8,totalGold_9,towers_lost_team100,towers_lost_team200,trueDamageDoneToChampions_1,trueDamageDoneToChampions_10,trueDamageDoneToChampions_2,trueDamageDoneToChampions_3,trueDamageDoneToChampions_4,trueDamageDoneToChampions_5,trueDamageDoneToChampions_6,trueDamageDoneToChampions_7,trueDamageDoneToChampions_8,trueDamageDoneToChampions_9,ward_player_0,ward_player_1,ward_player_10,ward_player_2,ward_player_3,ward_player_4,ward_player_5,ward_player_6,ward_player_7,ward_player_8,ward_player_9,winningTeam,xp_1,xp_10,xp_2,xp_3,xp_4,xp_5,xp_6,xp_7,xp_8,xp_9,assistsplayer_4,baron_team200,deathsplayer_10,deathsplayer_7,dragon_team100,inhibs_lost_team200,riftherald_team100,baron_team100,killsplayer_0,team_deathsplayer_100,team_deathsplayer_200,team_goldPerSecond_100,team_goldPerSecond_200,team_jungleMinionsKilled_100,team_jungleMinionsKilled_200,team_killsplayer_100,team_killsplayer_200,team_level_100,team_level_200,team_magicDamageDoneToChampions_100,team_magicDamageDoneToChampions_200,team_minionsKilled_100,team_minionsKilled_200,team_physicalDamageDoneToChampions_100,team_physicalDamageDoneToChampions_200,team_timeEnemySpentControlled_100,team_timeEnemySpentControlled_200,team_totalDamageDoneToChampions_100,team_totalDamageDoneToChampions_200,team_totalGold_100,team_totalGold_200,team_trueDamageDoneToChampions_100,team_trueDamageDoneToChampions_200,team_ward_player_100,team_ward_player_200,team_assistsplayer_100,team_assi

In [29]:
df.size

150384

In [30]:
df.shape

(723, 208)

In [36]:
df.columns

Index(['assistsplayer_1', 'assistsplayer_10', 'assistsplayer_2',
       'assistsplayer_3', 'assistsplayer_5', 'assistsplayer_6',
       'assistsplayer_7', 'assistsplayer_8', 'assistsplayer_9',
       'currentGold_1',
       ...
       'team_totalGold_100', 'team_totalGold_200',
       'team_trueDamageDoneToChampions_100',
       'team_trueDamageDoneToChampions_200', 'team_ward_player_100',
       'team_ward_player_200', 'team_assistsplayer_100',
       'team_assistsplayer_200', 'team_xp_100', 'team_xp_200'],
      dtype='object', length=208)

In [37]:
# df.to_csv('bryant_games.csv')

In [43]:
df.winningTeam.value_counts()

200.0    369
100.0    354
Name: winningTeam, dtype: int64